# Purpose
define basic functions

In [ ]:
%pip install -q --force-reinstall git+https://github.com/Surpris/BraggPy.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import braggpy

%matplotlib inline

## make lattice points

In [ ]:
d = 6.0  # $\AA$
lattice_type = "fcc"
ind_max = 10
coors_origin = braggpy.make_lattice_points(d, lattice_type=lattice_type, ind_min=-ind_max, ind_max=ind_max)

In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection="3d")
ax.scatter3D(coors_origin[:,0], coors_origin[:,1], coors_origin[:,2], color="aqua", edgecolor="k", alpha=0.5)
ax.set_xlabel(r"x ($\AA$)", fontsize=14)
ax.set_ylabel(r"y ($\AA$)", fontsize=14)
ax.set_zlabel(r"z ($\AA$)", fontsize=14)

## choose coordinates inside some shapes

In [ ]:
shape_name = "sphere"
R = 30.
coor_inside = braggpy.is_inside(coors_origin, R, shape_name)

In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection="3d")
ax.scatter3D(
    coor_inside[:,0], coor_inside[:,1], coor_inside[:,2],
    color="aqua", edgecolor="k", alpha=0.5
)
ax.set_xlabel(r"x ($\AA$)", fontsize=14)
ax.set_ylabel(r"y ($\AA$)", fontsize=14)
ax.set_zlabel(r"z ($\AA$)", fontsize=14)

## generate momentum coordinates

In [ ]:
hv = 12.  # [keV]
qmax = 3.5
dq = 0.05

res = braggpy.generate_momentum(hv, qmax, dq)

In [ ]:
extent = [
    res["qmin"] - res["dqx"] / 2.0, res["qmax"] + res["dqx"] / 2.0,
    res["qmin"] - res["dqy"] / 2.0, res["qmax"] + res["dqy"] / 2.0
]  # (left, right, bottom, top)

plt.figure(figsize=(6,5))
plt.imshow(res["qzz"], origin="lower", extent=extent)
plt.colorbar()
plt.xlabel("$q_x (1/\AA$)")
plt.ylabel("$q_y (1/\AA$)")

## euler rotation

In [ ]:
miller_idx = [1, 1, 1]

euler = np.rad2deg(braggpy.calc_euler_hkl(d, 2.*np.pi/res["wavelength"], *miller_idx))
euler

In [ ]:
# euler = [45, 60., 0.]

coors_euler = braggpy.euler_rotate(coors_origin, euler, 1)
coor_inside = braggpy.is_inside(coors_euler, R, shape_name)

In [ ]:
fig = plt.figure(figsize=(12, 5))

# Original atoms
ax = fig.add_subplot(121, projection="3d")
ax.scatter3D(
    coors_euler[:,0], coors_euler[:,1], coors_euler[:,2],
    color="aqua", edgecolor="k", alpha=0.5
)
ax.set_xlabel(r"x ($\AA$)", fontsize=14)
ax.set_ylabel(r"y ($\AA$)", fontsize=14)
ax.set_zlabel(r"z ($\AA$)", fontsize=14)

# atoms in a sphere
ax = fig.add_subplot(122, projection="3d")
ax.scatter3D(
    coor_inside[:,0], coor_inside[:,1], coor_inside[:,2],
    color="aqua", edgecolor="k", alpha=0.5
)
ax.set_xlabel(r"x ($\AA$)", fontsize=14)
ax.set_ylabel(r"y ($\AA$)", fontsize=14)
ax.set_zlabel(r"z ($\AA$)", fontsize=14)

## calculate modulus

In [ ]:
F = braggpy.calc_modulus(coor_inside, res["qxx"], res["qyy"], res["qzz"])

In [ ]:
d_lattice = braggpy.calculate_lattice_distance(d, *miller_idx)
q_spot = 2. * np.pi / d_lattice

In [ ]:
extent = [
    res["qmin"] - res["dqx"] / 2.0, res["qmax"] + res["dqx"] / 2.0,
    res["qmin"] - res["dqy"] / 2.0, res["qmax"] + res["dqy"] / 2.0
]  # (left, right, bottom, top)

# show the modulus
plt.figure(figsize=(6,5))
plt.imshow(np.abs(F), origin="lower", extent=extent)

plt.xlabel("$q_x (1/\AA$)")
plt.ylabel("$q_y (1/\AA$)")

# plot a circle with a radius of q_spot

theta = np.arange(0., 2.*np.pi, np.pi/100) 
plt.plot(q_spot * np.sin(theta), q_spot * np.cos(theta), "-")